# Exercise 02: SVD/PCA on MNIST Digits

In this exercise,
you'll explore the use of Singular Value Decomposition (SVD)
to perform Principle Component Analysis (PCA)
on the MNIST handwritten digits dataset.

As usual, let's start by installing the needed packages:

In [ ]:
import sys
!{sys.executable} -m pip install -U pip
!{sys.executable} -m pip install -U scikit-learn matplotlib

Now let's load the MNIST dataset:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
dataset = datasets.load_digits()

_, axes = plt.subplots(nrows=1, ncols=20, figsize=(10, 3))
for ax, image, label in zip(axes, dataset.images, dataset.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title("%i" % label)

We have 1797 examples of 8x8 pixel images:

In [ ]:
dataset.images.shape

**How many dimensions does each image have?**

(Hint #1: The answer isn't 2.)

(Hint #2: How many numbers does it take to represent one image?)

YOUR ANSWER HERE.

We're going to "flatten" the 8x8 pixel images into 64 pixel vectors to aid our processing.
Note the transpose (`.T`); each image vector is now a column vector.

In [ ]:
N_samples = len(dataset.images)
data = dataset.images.reshape((N_samples, 64)).T
data.shape

Let's take a compare the tranpose of the first data vector
(displayed as a row instead of a column)
to the first 8x8 dataset image.
Make sure you understand what you're looking at!

In [ ]:
print('data[:,0]=\n{}'.format(data[:,0]))
print('dataset.images[0]=\n{}'.format(dataset.images[0]))

**Why do we need to normalize our data before performing SVD?**

YOUR ANSWER HERE.

Let's go ahead and normalize.  We need to keep the mean and stddev info for later use.

In [ ]:
data_mean = np.mean(data, axis=1).reshape((64,1))
#print('data_mean = \n{}'.format(data_mean))

data_stddev = np.std(data, axis=1).reshape((64,1))
data_stddev[data_stddev == 0] = 0.001
#print('data_stddev = \n{}'.format(data_stddev))

data_normalized = (data - data_mean)/data_stddev

Inspect the following code and its output:

In [ ]:
print(np.mean(data_normalized, axis=1))
print(np.std(data_normalized, axis=1))

**Do these values make sense?  Why or why not?  Do you see anything unexpected?**

YOUR ANSWER HERE.

Now that we have normalized data,
we can construct the covariance matrix.

**What shape do you expect the covariance matrix to have?  Why?**

YOUR ANSWER HERE.

In [ ]:
C = np.dot(data_normalized,data_normalized.T)
C.shape

Before you run the code below,
stop and think about what we're doing:
we want to reduce the dimensionality of the MNIST dataset
to something more "manageable".

**What is the minimum number of dimensions we can reasonably expect for this dataset?  Why?**

YOUR ANSWER HERE.

Let's perform SVD on the covariance matrix,
and inspect the diagonal, `S`, of the singular value matrix:

In [ ]:
U,S,Vh = np.linalg.svd(C)
print('S = \n{}'.format(S))
plt.plot(S)
plt.show();

**Based on the values and scree plot above, how many meaningful dimensions does this MNIST data have?  How did you come to that conclusion?**

YOUR ANSWER HERE.

**Based on the values and scree plot above, how many dimensions can we *definitely* get rid of?**

YOUR ANSWER HERE.

**What do the dimensions we can eliminate say about the input data values?**

YOUR ANSWER HERE.

Below I've "inverted" the scree plot
to show the percentage of information retained
versus number of dimensions retained.

In [ ]:
plt.plot(np.cumsum(S)/np.sum(S))
plt.show()

Let's retain only 5 dimensions and take a look at the results.

First, we'll project our data onto the best 5D approximation of our covariance matrix:

In [ ]:
ND = 5

P = np.dot(np.diagflat(S[0:ND]), Vh[0:ND,:])
compressed_data = np.dot(P,data_normalized)
compressed_data.shape

**Explain the shape of `compressed_data` shown above.**

YOUR ANSWER HERE.

Our `compressed_data` currently lives in a 5D space with the basis determined by SVD.
We'd like to bring it back into our original image space and coordinate system.

Unfortunately, the matrix `P` is singular, so we can't invert it.
Instead, we'll use the "pseudo-inverse" as the next best thing:

In [ ]:
UnP = np.linalg.pinv(P)
UnP.shape

Now we'll use our "unpack" (`UnP`) matrix to bring our `compressed_data` back into the original input space.

Note that we need to undo the data normalization, because that wasn't accounted for by the SVD process.

In [ ]:
unpacked_data = np.dot(UnP,compressed_data)*data_stddev + data_mean
unpacked_images = np.reshape(unpacked_data.T, (n_samples, 8, 8))
unpacked_images.shape

Let's see how well 5 dimensions representated our data.

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=20, figsize=(10, 3))
for ax, image, label in zip(axes, unpacked_images, dataset.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title("%i" % label)

**What are your thoughts on these images?  Are 5 dimensions enough?  Why do some numbers appear reasonably clear while others are garbled?**

YOUR ANSWER HERE.

Take a look at the first unpacked image values versus the original image values:

In [ ]:
print('unpacked_images[0] =\n{}'.format(unpacked_images[0]))
print('dataset.images[0] = \n{}'.format(dataset.images[0]))

Let's try the same steps again, but this time we'll retain all 64 dimensions.

In [ ]:
ND = 64

P = np.dot(np.diagflat(S[0:ND]), Vh[0:ND,:])
compressed_data = np.dot(P,data_normalized)
compressed_data.shape

In [ ]:
UnP = np.linalg.pinv(P)

unpacked_data = np.dot(UnP,compressed_data)*data_stddev + data_mean
unpacked_images = np.reshape(unpacked_data.T, (n_samples, 8, 8))

_, axes = plt.subplots(nrows=1, ncols=20, figsize=(10, 3))
for ax, image, label in zip(axes, unpacked_images, dataset.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title("%i" % label)

For comparison, let's look at the original dataset images.

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=20, figsize=(10, 3))
for ax, image, label in zip(axes, dataset.images, dataset.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title("%i" % label)

**What do you notice?  Why is that the case?**

YOUR ANSWER HERE.

Let's compare the unpacked/original image values again:

In [ ]:
print('unpacked_images[0] =\n{}'.format(unpacked_images[0]))
print('dataset.images[0] = \n{}'.format(dataset.images[0]))

**How well did 64 dimensions approximate our original dataset?  Why?**

YOUR ANSWER HERE.

It would seem that there is some middle ground between 5D and 64D that would serve our purpose.  Let's try to find it. 

In [ ]:
for ND in range(4,65,4):    
    P = np.dot(np.diagflat(S[0:ND]), Vh[0:ND,:])
    compressed_data = np.dot(P,data_normalized)
    UnP = np.linalg.pinv(P)

    unpacked_data = np.dot(UnP,compressed_data)*data_stddev + data_mean
    unpacked_images = np.reshape(unpacked_data.T, (n_samples, 8, 8))

    _, axes = plt.subplots(nrows=1, ncols=20, figsize=(10, 3))
    for ax, image, label in zip(axes, unpacked_images, dataset.target):
        ax.set_axis_off()
        ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
        ax.set_title("%i" % ND)

**Based on the above plots, how many dimensions would you retain?  How did you come to that conclusion?**

YOUR ANSWER HERE.

**What did you think of this exercise?**

YOUR ANSWER HERE.